# Load required packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

# Load data

In [2]:
path = "/Users/tian/Downloads/lish-moa/"

In [3]:
train_df = pd.read_csv(path+'train_features.csv',index_col=['sig_id'])
train_target_df = pd.read_csv(path+'train_targets_scored.csv',index_col=['sig_id'])

In [4]:
test_df = pd.read_csv(path+'test_features.csv',index_col=['sig_id'])

In [5]:
train_df.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [6]:
train_target_df.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_000779bfc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_000a6266a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_0015fd391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_001626bd3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
for i in range(3):
    print(f"{train_df.iloc[:,i].value_counts()}'\n'")

trt_cp         21948
ctl_vehicle     1866
Name: cp_type, dtype: int64'
'
48    8250
72    7792
24    7772
Name: cp_time, dtype: int64'
'
D1    12147
D2    11667
Name: cp_dose, dtype: int64'
'


# Preprocessing

In [8]:
train_df.isna().sum().sum()

0

In [9]:
def preprocess(df):
    df = df.copy()
    df['cp_type'] = df['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})
    df['cp_dose'] = df['cp_dose'].map({'D1':0, 'D2':1})
    return df

train = preprocess(train_df)
test = preprocess(test_df)

In [10]:
train_target_df = train_target_df.loc[train['cp_type']==0]
train = train.loc[train['cp_type']==0]

In [11]:
train = train.merge(train_target_df.iloc[:,0],left_index=True,right_index=True,how='inner')


In [12]:
shuffle_df = train.sample(frac=1, random_state=1)
ratio = int(len(shuffle_df)*0.75)
Xtrain = shuffle_df.iloc[:ratio]
Xtest = shuffle_df.iloc[ratio:]
print(f"{Xtrain.shape}, {Xtest.shape}")

(16461, 876), (5487, 876)


In [13]:
Scaler = MinMaxScaler()
train_scaled = Scaler.fit_transform(Xtrain)
test_scaled = Scaler.transform(Xtest)

# Baseline model

In [14]:
clf = RandomForestClassifier(max_features=8, random_state=0)
clf.fit(train_scaled[:,:-1],train_scaled[:,-1])

RandomForestClassifier(max_features=8, random_state=0)

In [15]:
clf.score(test_scaled[:,:-1],test_scaled[:,-1])

0.9992710041917259

In [16]:
# for i in range(train_target_df.shape[0]):
#     print(f"{(train_target_df.iloc[i,]==1).sum()}")